# Import

In [35]:
import pandas as pd

In [36]:
df=pd.read_csv("./before/summer_player.csv",encoding="utf-8",usecols=["Player","Team","Pos","KP","KDA","GD10","XPD10","CSD10","DPM","DMG%","WPM","CWPM","W%"],sep=",")
df['KP']=df['KP'].apply(lambda x:str(x).split('%')[0])
df['DMG%']=df['DMG%'].apply(lambda x:str(x).split('%')[0])
df['W%']=df['W%'].apply(lambda x:str(x).split('%')[0])
df=df.astype({'KP':'float64','DMG%':'float64','W%':'float64'})
df.head(5)

,Player,Team,Pos,W%,KDA,KP,GD10,XPD10,CSD10,DPM,DMG%,WPM,CWPM
0,Aiming,KT Rolster,ADC,56.0,6.0,71.5,164,-7,5.7,630,30.3,0.55,0.16
1,Aria,KT Rolster,Middle,33.0,4.1,74.2,-190,-93,-5.9,507,24.7,0.78,0.33
2,Bdd,Nongshim RedForce,Middle,33.0,3.3,69.4,-7,-16,0.8,562,28.7,0.50,0.26
3,BeryL,DRX,Support,49.0,2.6,73.1,147,151,7.4,170,8.2,1.44,0.47
4,Bible,DWG KIA,Support,20.0,1.6,67.3,55,54,2.2,130,6.5,1.61,0.47


## preprocess

In [37]:
def minmax_norm(df_input):
    return (df_input - df_input.min()) / ( df_input.max() - df_input.min())

def normalize(df_input):
    return (df_input - df_input.mean()) / df_input.std()
def give_level(x):
    if x>=1.28:
        return 90
    elif x>=0.84:
        return 80
    elif x>=0.25:
        return 70
    elif x>=-0.53:
        return 60
    else:
        return 50

In [38]:
def give_tier(x):
    ## 10퍼
    if x>=1.28:
        return "UR"
    ## 20퍼
    elif x>=0.84:
        return "SR"
    #40퍼
    elif x>=0.25:
        return "R"
    #70퍼
    elif x>=-0.53:
        return "N"
    else:
        return "C"

In [39]:
def player_preprocess(df):
    gold = minmax_norm(df['GD10'])
    XP =minmax_norm(df['XPD10'])
    CS = minmax_norm(df['CSD10'])

    laning = (gold + XP + CS)/3
    laning = normalize(laning)
    laning = pd.DataFrame({'lane_status':laning})
    laning = laning['lane_status'].apply(give_level)
    df = pd.concat([df,laning],axis=1)

    
    
    KP = minmax_norm(df['KP'])
    KDA = minmax_norm(df['KDA'])
    WPM = minmax_norm(df['WPM'])
    CWPM = minmax_norm(df['CWPM'])
    W = minmax_norm(df['W%'])
    operation = 0.4*W + 0.1 * KP + 0.1 * KDA + 0.2 * WPM  + 0.2 * CWPM
    operation = normalize(operation)
    operation = pd.DataFrame({'operation_status':operation})
    operation = operation['operation_status'].apply(give_level)
    df = pd.concat([df,operation],axis=1)
    
    
    
    
    DPM = minmax_norm(df['DPM'])
    DMG = minmax_norm(df['DMG%'])
    
    fight = DPM + KDA
    fight = normalize(fight)
    fight = pd.DataFrame({'fight_status':fight})
    fight = fight['fight_status'].apply(give_level)
    df = pd.concat([df,fight],axis=1)
    
    
    
    total_ability = ((laning + operation + fight) / 3)
    total_ability = normalize(total_ability)
    grade = pd.DataFrame({'grade':total_ability})
    grade = grade['grade'].apply(give_tier)
    df = pd.concat([df,grade],axis=1)

    
    df = df.drop(columns = ["KP","KDA","GD10","XPD10","CSD10","DPM","DMG%","WPM","CWPM","W%"])
    df = df.rename(columns = {'Player':'nick_name'})
    df = df.rename(columns = {'Pos':'position'})
    df = df.rename(columns = {'Team':'base_team_id'})

    return df

## top

In [40]:
top=df[df["Pos"]=="Top"]

In [41]:
top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 5 to 61
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  11 non-null     object 
 1   Team    11 non-null     object 
 2   Pos     11 non-null     object 
 3   W%      11 non-null     float64
 4   KDA     11 non-null     float64
 5   KP      11 non-null     float64
 6   GD10    11 non-null     int64  
 7   XPD10   11 non-null     int64  
 8   CSD10   11 non-null     float64
 9   DPM     11 non-null     int64  
 10  DMG%    11 non-null     float64
 11  WPM     11 non-null     float64
 12  CWPM    11 non-null     float64
dtypes: float64(7), int64(3), object(3)
memory usage: 1.2+ KB


In [42]:
top

,Player,Team,Pos,W%,KDA,KP,GD10,XPD10,CSD10,DPM,DMG%,WPM,CWPM
5,Canna,Nongshim RedForce,Top,33.0,2.6,59.4,123,80,2.1,496,25.7,0.41,0.19
15,Doran,Gen.G,Top,88.0,4.2,56.2,62,72,1.2,547,24.5,0.41,0.24
16,Dove,Liiv SANDBOX,Top,66.0,4.1,63.0,-281,-162,-8.5,459,22.1,0.40,0.23
18,DuDu,Hanwha Life Esports,Top,26.0,1.7,61.8,56,12,3.8,464,27.4,0.33,0.18
33,Kiin,Kwangdong Freecs,Top,31.0,1.9,61.0,-4,-37,1.3,462,27.3,0.42,0.23
34,Kingen,DRX,Top,49.0,3.1,60.7,-190,-104,-3.2,496,24.0,0.41,0.24
39,Morgan,Fredit BRION,Top,31.0,2.3,59.7,-109,-53,-0.5,409,23.6,0.42,0.25
40,Nuguri,DWG KIA,Top,59.0,3.2,57.5,157,47,1.2,518,24.2,0.35,0.20
48,Rascal,KT Rolster,Top,56.0,3.4,63.8,-5,69,-1.2,451,21.9,0.36,0.21
53,Sw0rd,Fredit BRION,Top,0.0,1.2,52.4,-197,-242,-6.8,522,29.9,0.39,0.22


In [43]:
top = player_preprocess(top)
top

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade
5,Canna,Nongshim RedForce,Top,80,60,60,R
15,Doran,Gen.G,Top,70,90,90,UR
16,Dove,Liiv SANDBOX,Top,50,80,70,R
18,DuDu,Hanwha Life Esports,Top,70,50,50,C
33,Kiin,Kwangdong Freecs,Top,60,60,50,C
34,Kingen,DRX,Top,50,70,70,N
39,Morgan,Fredit BRION,Top,60,70,50,N
40,Nuguri,DWG KIA,Top,70,60,70,R
48,Rascal,KT Rolster,Top,70,60,60,N
53,Sw0rd,Fredit BRION,Top,50,50,50,C


## middle

In [44]:
middle=df[df["Pos"]=="Middle"]

In [45]:
middle

,Player,Team,Pos,W%,KDA,KP,GD10,XPD10,CSD10,DPM,DMG%,WPM,CWPM
1,Aria,KT Rolster,Middle,33.0,4.1,74.2,-190,-93,-5.9,507,24.7,0.78,0.33
2,Bdd,Nongshim RedForce,Middle,33.0,3.3,69.4,-7,-16,0.8,562,28.7,0.50,0.26
8,Chovy,Gen.G,Middle,88.0,7.0,64.5,196,115,5.0,599,26.7,0.51,0.16
9,Clozer,Liiv SANDBOX,Middle,66.0,4.7,65.7,3,68,1.0,589,27.6,0.41,0.17
21,Faker,T1,Middle,70.0,4.2,65.1,90,-13,0.8,528,25.5,0.46,0.21
22,FATE,Kwangdong Freecs,Middle,31.0,4.0,71.9,-67,-27,0.2,421,24.6,0.45,0.24
30,Karis,Hanwha Life Esports,Middle,26.0,2.0,68.5,-207,-85,-4.0,425,24.5,0.38,0.20
35,Lava,Fredit BRION,Middle,26.0,2.8,74.2,-6,-70,-2.9,514,30.1,0.40,0.17
51,ShowMaker,DWG KIA,Middle,59.0,6.0,67.9,5,45,0.4,499,23.0,0.51,0.25
57,VicLa,KT Rolster,Middle,61.0,4.0,68.2,1,57,-1.7,555,26.6,0.50,0.29


In [46]:
middle = player_preprocess(middle)
middle

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade
1,Aria,KT Rolster,Middle,50,90,60,N
2,Bdd,Nongshim RedForce,Middle,60,60,60,N
8,Chovy,Gen.G,Middle,90,80,90,UR
9,Clozer,Liiv SANDBOX,Middle,70,60,80,R
21,Faker,T1,Middle,70,70,60,N
22,FATE,Kwangdong Freecs,Middle,60,50,50,C
30,Karis,Hanwha Life Esports,Middle,50,50,50,C
35,Lava,Fredit BRION,Middle,50,50,50,C
51,ShowMaker,DWG KIA,Middle,70,70,70,R
57,VicLa,KT Rolster,Middle,70,80,60,R


## jungle

In [47]:
jungle=df[df["Pos"]=="Jungle"]

In [48]:
jungle

,Player,Team,Pos,W%,KDA,KP,GD10,XPD10,CSD10,DPM,DMG%,WPM,CWPM
6,Canyon,DWG KIA,Jungle,59.0,3.9,72.3,-93,31,1.1,340,15.2,0.46,0.39
10,Croco,Liiv SANDBOX,Jungle,66.0,3.8,70.6,-2,-16,-1.5,302,14.3,0.31,0.23
11,Cuzz,KT Rolster,Jungle,56.0,3.9,69.2,61,5,1.2,283,13.6,0.38,0.34
17,Dread,Nongshim RedForce,Jungle,37.0,3.5,70.7,-12,-66,-2.7,260,12.9,0.45,0.41
20,Ellim,Kwangdong Freecs,Jungle,31.0,2.3,72.5,-171,-167,-5.6,187,11.1,0.48,0.45
28,Juhan,DRX,Jungle,33.0,1.6,60.0,-174,-173,-5.7,230,11.9,0.53,0.57
41,Oner,T1,Jungle,70.0,4.4,74.3,203,143,2.5,294,14.0,0.38,0.25
42,OnFleek,Hanwha Life Esports,Jungle,28.0,1.7,68.0,-107,-128,-1.6,248,14.3,0.44,0.37
43,Peanut,Gen.G,Jungle,88.0,5.8,65.2,240,185,4.4,262,11.6,0.45,0.30
46,Pyosik,DRX,Jungle,50.0,3.6,72.5,0,130,3.5,266,12.5,0.50,0.45


In [49]:
jungle = player_preprocess(jungle)
jungle

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade
6,Canyon,DWG KIA,Jungle,60,80,90,SR
10,Croco,Liiv SANDBOX,Jungle,60,60,80,N
11,Cuzz,KT Rolster,Jungle,70,60,70,N
17,Dread,Nongshim RedForce,Jungle,60,60,70,N
20,Ellim,Kwangdong Freecs,Jungle,50,70,50,C
28,Juhan,DRX,Jungle,50,70,50,C
41,Oner,T1,Jungle,90,70,80,UR
42,OnFleek,Hanwha Life Esports,Jungle,50,50,60,C
43,Peanut,Gen.G,Jungle,90,90,90,UR
46,Pyosik,DRX,Jungle,80,80,70,SR


## adc

In [50]:
adc=df[df["Pos"]=="ADC"]

In [51]:
adc

,Player,Team,Pos,W%,KDA,KP,GD10,XPD10,CSD10,DPM,DMG%,WPM,CWPM
0,Aiming,KT Rolster,ADC,56.0,6.0,71.5,164,-7,5.7,630,30.3,0.55,0.16
7,Cheoni,Hanwha Life Esports,ADC,14.0,2.0,72.7,-447,-184,-9.6,368,25.5,0.42,0.16
12,Deft,DRX,ADC,49.0,3.7,71.4,-27,-64,-5.8,581,27.9,0.66,0.19
14,deokdam,DWG KIA,ADC,59.0,4.9,73.3,-29,-50,1.2,616,28.6,0.46,0.13
23,Gamin,Fredit BRION,ADC,0.0,1.2,59.1,-222,-142,9.7,232,17.0,0.41,0.11
24,Ghost,Nongshim RedForce,ADC,33.0,3.1,69.0,-151,-20,-5.1,502,25.0,0.57,0.19
25,Gumayusi,T1,ADC,70.0,4.5,71.1,52,10,4.8,515,24.5,0.55,0.21
26,Hena,Fredit BRION,ADC,28.0,2.9,71.6,-25,51,4.6,447,25.4,0.47,0.15
45,Prince,Liiv SANDBOX,ADC,66.0,6.2,73.3,211,185,7.8,624,28.9,0.62,0.15
49,Ruler,Gen.G,ADC,88.0,7.6,71.5,172,88,-0.7,672,29.5,0.65,0.25


In [52]:
adc = player_preprocess(adc)
adc

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade
0,Aiming,KT Rolster,ADC,70,70,80,SR
7,Cheoni,Hanwha Life Esports,ADC,50,50,50,C
12,Deft,DRX,ADC,60,70,60,N
14,deokdam,DWG KIA,ADC,60,60,70,N
23,Gamin,Fredit BRION,ADC,60,50,50,C
24,Ghost,Nongshim RedForce,ADC,50,60,60,C
25,Gumayusi,T1,ADC,70,70,60,R
26,Hena,Fredit BRION,ADC,70,50,60,N
45,Prince,Liiv SANDBOX,ADC,90,70,80,UR
49,Ruler,Gen.G,ADC,70,90,90,UR


## support

In [53]:
sup=df[df["Pos"]=="Support"]

In [54]:
sup

,Player,Team,Pos,W%,KDA,KP,GD10,XPD10,CSD10,DPM,DMG%,WPM,CWPM
3,BeryL,DRX,Support,49.0,2.6,73.1,147,151,7.4,170,8.2,1.44,0.47
4,Bible,DWG KIA,Support,20.0,1.6,67.3,55,54,2.2,130,6.5,1.61,0.47
13,Delight,Fredit BRION,Support,26.0,2.3,70.3,-100,-47,-5.5,121,7.1,1.83,0.54
19,Effort,Nongshim RedForce,Support,27.0,2.3,71.4,-7,-7,6.7,148,7.8,1.68,0.54
27,Hoit,Kwangdong Freecs,Support,33.0,3.0,73.3,134,133,10.5,168,9.7,1.59,0.51
29,Kael,Liiv SANDBOX,Support,66.0,4.6,78.5,-81,-120,-4.1,148,7.1,1.70,0.54
31,Kellin,DWG KIA,Support,64.0,5.0,78.0,-58,-123,-5.9,202,9.4,1.82,0.53
32,Keria,T1,Support,70.0,4.5,68.0,55,56,-0.8,189,9.1,1.80,0.51
36,Lehends,Gen.G,Support,88.0,5.6,71.5,25,100,2.8,180,7.8,1.56,0.45
37,Life,KT Rolster,Support,56.0,4.3,77.1,4,51,-7.0,162,7.9,1.93,0.42


In [55]:
sup = player_preprocess(sup)
sup

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade
3,BeryL,DRX,Support,90,60,60,R
4,Bible,DWG KIA,Support,70,50,50,C
13,Delight,Fredit BRION,Support,50,60,50,C
19,Effort,Nongshim RedForce,Support,70,60,50,N
27,Hoit,Kwangdong Freecs,Support,90,60,60,R
29,Kael,Liiv SANDBOX,Support,50,90,70,R
31,Kellin,DWG KIA,Support,50,90,90,SR
32,Keria,T1,Support,70,80,80,SR
36,Lehends,Gen.G,Support,70,80,90,UR
37,Life,KT Rolster,Support,60,70,70,N


In [56]:
final=pd.concat([top,jungle,middle,adc,sup],ignore_index=True)

In [57]:
final

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade
0,Canna,Nongshim RedForce,Top,80,60,60,R
1,Doran,Gen.G,Top,70,90,90,UR
2,Dove,Liiv SANDBOX,Top,50,80,70,R
3,DuDu,Hanwha Life Esports,Top,70,50,50,C
4,Kiin,Kwangdong Freecs,Top,60,60,50,C
...,...,...,...,...,...,...,...
57,Life,KT Rolster,Support,60,70,70,N
58,Moham,Kwangdong Freecs,Support,80,50,50,N
59,Peter,Nongshim RedForce,Support,50,50,50,C
60,SnowFlower,Nongshim RedForce,Support,70,60,60,N


In [58]:
final['id']=final.index+1

In [59]:
final['real_name']="name"

In [60]:
final

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade,id,real_name
0,Canna,Nongshim RedForce,Top,80,60,60,R,1,name
1,Doran,Gen.G,Top,70,90,90,UR,2,name
2,Dove,Liiv SANDBOX,Top,50,80,70,R,3,name
3,DuDu,Hanwha Life Esports,Top,70,50,50,C,4,name
4,Kiin,Kwangdong Freecs,Top,60,60,50,C,5,name
...,...,...,...,...,...,...,...,...,...
57,Life,KT Rolster,Support,60,70,70,N,58,name
58,Moham,Kwangdong Freecs,Support,80,50,50,N,59,name
59,Peter,Nongshim RedForce,Support,50,50,50,C,60,name
60,SnowFlower,Nongshim RedForce,Support,70,60,60,N,61,name


In [61]:
def change_team(elem):
    if elem=="T1":
        return 1;
    if elem=="Gen.G":
        return 2;
    if elem=="DRX":
        return 3;
    if elem=="DWG KIA":
        return 4;
    if elem=="Fredit BRION":
        return 5;
    if elem=="Kwangdong Freecs":
        return 6;
    if elem=="Hanwha Life Esports":
        return 7;
    if elem=="KT Rolster":
        return 8;
    if elem=="Liiv SANDBOX":
        return 9;
    if elem=="Nongshim RedForce":
        return 10;
    

In [62]:
final['base_team_id'] = final['base_team_id'].apply(change_team)

In [63]:
final.head()

,nick_name,base_team_id,position,lane_status,operation_status,fight_status,grade,id,real_name
0,Canna,10,Top,80,60,60,R,1,name
1,Doran,2,Top,70,90,90,UR,2,name
2,Dove,9,Top,50,80,70,R,3,name
3,DuDu,7,Top,70,50,50,C,4,name
4,Kiin,6,Top,60,60,50,C,5,name


In [64]:
final.to_csv("./csv/player_no_real_name.csv",index=False,sep=";",encoding="utf-8")